# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 07**: Structured Streaming with Files 

**Date**: October 10th 2025

**Student Name**: 

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on Structured Streaming (files)") \
    .master("spark://spark-master:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")
# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/10 05:01:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
!pwd

/opt/spark/work-dir/labs/lab07


In [ ]:
from pcamarillor.spark_utils import SparkUtils
import re
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.types import StructType, StructField, StringType

logs_schema = StructType([
    StructField("value", StringType(), True)
])
logs_df = spark.readStream \
    .format("text") \
    .schema(logs_schema) \
    .load("/opt/spark/work-dir/data/custom-logs/")
logs_parsed_df = logs_df \
    .withColumn("timestamp", regexp_extract(col("value"), r"^(\S+ \S+)", 1)) \
    .withColumn("endpoint", regexp_extract(col("value"), r"- (/\S+)", 1)) \
    .withColumn("status_code", regexp_extract(col("value"), r"- (\d{3})$", 1))

query = logs_parsed_df.writeStream \
            .outputMode("append") \
            .format("console") \
            .start()

query.awaitTermination(30) # Wait 30 seconds before closing the stream

-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+-------------------+--------------+-----------+
|               value|          timestamp|      endpoint|status_code|
+--------------------+-------------------+--------------+-----------+
|2025-10-09 22:59:...|2025-10-09 22:59:52|/api/v1/upload|        404|
|2025-10-09 22:59:...|2025-10-09 22:59:52|       /logout|        404|
|2025-10-09 22:59:...|2025-10-09 22:59:52|    /dashboard|        503|
|2025-10-09 22:59:...|2025-10-09 22:59:52|  /api/v1/data|        200|
|2025-10-09 22:59:...|2025-10-09 22:59:52|       /logout|        503|
|2025-10-09 22:59:...|2025-10-09 22:59:52|/api/v1/upload|        503|
|2025-10-09 22:59:...|2025-10-09 22:59:52|       /logout|        503|
|2025-10-09 22:59:...|2025-10-09 22:59:52|/api/v1/upload|        200|
|2025-10-09 22:59:...|2025-10-09 22:59:52|  /api/v1/data|        500|
|2025-10-09 22:59:...|2025-10-09 22:59:52|  /api/v1/data|      

In [ ]:
sc.stop()